## Appendix B: Data Types' Internal Representation

### Integer Numbers

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/appendixB/uint8.png?raw=True)

<center>Figure B.1 - Internal representation of an unsigned 8-bit integer (UINT8)</center>

$$
\Large
x = \sum_{j=1}^{n_B}{b_j2^{j-1}}
\\
\Large
\begin{aligned}
&n_B = \text{number of bits}
\\
&b_j = j^{\text{th}}\text {from right to left}
\end{aligned}
$$

<center>Equation B.1 - Formula for computing an unsigned integer value from its bits</center>

In [60]:
def get_unsigned_number(bits):
    number = sum([int(bit)*2**(j-1) for j, bit in enumerate(bits[::-1], start=1)])
    return number

In [66]:
get_unsigned_number('10000011')

131

In [71]:
torch.iinfo(torch.uint8)

iinfo(min=0, max=255, dtype=uint8)

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/appendixB/int8.png?raw=True)
<center>Figure B.3 - Internal representation of a signed 8-bit integer (INT8)</center>

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/appendixB/int8_example.png?raw=True)
<center>Figure B.4 - Example of signed integer value and its bits</center>

$$
\Large
x = -b_{n_B}2^{n_B-1}+\sum_{j=1}^{n_B-1}{b_j2^{j-1}}
\\
\Large
\begin{aligned}
&n_B = \text{number of bits}
\\
&b_j = j^{\text{th}}\text {from right to left}
\\
&b_{n_B} = \text{left-most bit}
\end{aligned}
$$

<center>Equation B.2 - Formula for computing a signed integer value from its bits</center>

In [110]:
def get_number(bits, signed=True):
    nb = len(bits)
    sign = -signed*2**(nb-1)
    number = sum([int(bit)*2**(j-1) for j, bit in enumerate(bits[signed:][::-1], start=1)])
    return sign + number

In [114]:
get_number('10000011', signed=True)

-125

In [115]:
np.binary_repr(-125, width=8)

'10000011'

In [73]:
torch.iinfo(torch.int8)

iinfo(min=-128, max=127, dtype=int8)

### Floating Point Numbers

$$
\Large
\text{FP} = \underbrace{-1^S}_{\text{sign}}\underbrace{2^x}_{\text{exponent}}\underbrace{(1.0 + f)}_{\text{mantissa}}
$$
<center>Equation B.3 - Computing a floating-point number from its sign, mantissa, and exponent</center>

In [60]:
def to_fp(s, x, f):
    return (-1)**s * 2**x * (1 + f)

In [33]:
f_cte = 0
print(to_fp(s=0, x=-1, f=f_cte)) # = 1*(2**-1)*(1+0)
print(to_fp(s=0, x=-8, f=f_cte)) # = 1*(2**-8)*(1+0)

0.5
0.00390625


In [35]:
x_cte = -1
print(to_fp(s=0, x=x_cte, f=0)) # = 1*(2**-1)*(1+0)
print(to_fp(s=0, x=x_cte, f=.5))# = 1*(2**-1)*(1+.5)
print(to_fp(s=0, x=x_cte, f=.9999))# = 1*(2**-1)*(1+.99)

0.5
0.75
0.99995


In [40]:
new_x_cte = x_cte + 1
print(to_fp(s=0, x=new_x_cte, f=0))# = 1*(2**0)*(1+0)

1


In [44]:
print(to_fp(s=0, x=2, f=.55))# = 1*(2**2)*(1+.55)
print(to_fp(s=0, x=5, f=.15))# = 1*(2**5)*(1+.15)
print(to_fp(s=0, x=8, f=.5))# = 1*(2**8)*(1+.5)

6.2
36.8
384.0


![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/appendixB/bf16_example.png?raw=True)
<center>Figure B.5 - Example of sign, exponent, and mantissa, and their bits</center>

$$
\Large
f = \sum_{i=1}^{n_M}{m_i2^{-i}}
\\
\Large
x = \left(\sum_{j=1}^{n_E}{e_j2^{j-1}}\right) - b
\\
\Large
b = 2^{n_E-1}-1
\\
\Large
\begin{aligned}
&n_M = \text{number of bits in the mantissa}
\\
&m_i = i^{\text{th}}\text{ bit from left to right in the mantissa}
\\
&n_E = \text{number of bits in the exponent}
\\
&e_j = j^{\text{th}}\text {from right to left in the exponent}
\end{aligned}
$$
<center>Equation B.4 - Formula for computing f and x from the bits in the mantissa and the exponent</center>

In [69]:
def get_x(exponent):
    bias = 2**(len(exponent)-1)-1
    return sum([int(bit)*2**(j-1) for j, bit in enumerate(exponent[::-1], start=1)]) - bias

def get_f(mantissa):
    return sum([int(bit)*2**(-i) for i, bit in enumerate(mantissa, start=1)])

In [76]:
exponent = '10000011'
x = get_x(exponent)
x

4

In [71]:
mantissa = '0111011'
f = get_f(mantissa)
f

0.4609375

In [72]:
mantissa = '011101100000'
get_f(mantissa)

0.4609375

In [75]:
to_fp(0, x, f)

23.375

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/appendixB/bf16_diagram.png?raw=True)
<center>Figure B.6 - Internal representation of the BF16 data type</center>

$$
\Large
\text{BF16} = -1^S \left(1.0 + \sum_{i=1}^{7}{m_i2^{-i}}\right) 2^{\left(\sum_{j=1}^{8}{e_j2^{j-1}}\right)-127}
$$
<center>Equation B.5 - Formula for computing a BF16 value from its bits</center>

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/appendixB/types_comparison.png?raw=True)
<center>Figure B.7 - Comparing the internal representations of FP32, BF16, and FP16</center>

In [58]:
# Adapted from https://stackoverflow.com/questions/16444726/binary-representation-of-float-in-python-bits-not-hex  
import struct

def binary_fp32(num):
    bits = ''.join('{:0>8b}'.format(c) for c in struct.pack('!f', num))
    sign = bits[0]
    exponent = bits[1:9]
    mantissa = bits[9:]
    return {'sign': sign, 'exponent': exponent, 'mantissa': mantissa}

bits = binary_fp32(23.375)
bits

{'sign': '0', 'exponent': '10000011', 'mantissa': '01110110000000000000000'}

In [59]:
s = int(bits['sign'])
f = get_f(bits['mantissa'])
x = get_x(bits['exponent'])
to_fp(s, f, x)

23.375